In [7]:
text = "According to the New York Times, Warner Bros. wanted to delay Christopher Nolan’s #Tenet months ago due to the COVID-19 pandemic but Nolan refused to listen to any discussions. The studio kept the original release date to “keep [Nolan] happy"
verified = False

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import re, string
import keras
import nltk
import numpy as np
import tensorflow as tf
import scipy

stopwords = nltk.corpus.stopwords.words('english')

def tokenize_tweet(string_data:str):
    wordnet_lemmatizer = WordNetLemmatizer()
    tokenized = nltk.RegexpTokenizer('\w+')
    data = string_data.replace('\n', '')
    data = data.lower()
    data = re.sub('https?://\S+|www\.\S+', '', data)
    data = re.sub('[%s]' % re.escape(string.punctuation), '', data)
    data = tokenized.tokenize(data)
    data = [i for i in data if i not in stopwords]
    data = [wordnet_lemmatizer.lemmatize(word) for word in data]
    data = ' '.join(data)   
    return data

In [9]:
vectorizer = TfidfVectorizer()
train_file = "train.data.txt"
train_file = pd.read_csv('./%s.csv'%train_file,keep_default_na=False)
train_data = train_file['main_tweet']
train_data.replace('', np.nan, inplace=True)
train_data.dropna(inplace=True)
train_data = train_data.apply(lambda x: tokenize_tweet(x))
vectorizer.fit(train_data.tolist())
model = keras.models.load_model("tfidf.h5")
test_text = tokenize_tweet(text)
test_tfidf = vectorizer.transform([test_text])
test_tfidf = scipy.sparse.csr_matrix(test_tfidf).todense()
test_pd = pd.DataFrame(test_tfidf)
test_verified = pd.DataFrame({"verified":[int(verified)]})
test = pd.concat([test_pd, test_verified], axis=1)
test = tf.convert_to_tensor(test)
prediction = model.predict(test)
prediction = (prediction > 0.5).astype("int32")
prediction = np.ndarray.flatten(prediction)[0]
print(prediction)

2022-05-12 17:45:30.136725: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


0
